# Permutation Flow-Shop Scheduling Problem

This is a variant of the Flot-shop scheduling problem (FSSP) in which the sequence of jobs is the same in every machine.

$$
 \begin{align}
     \text{min} \quad & C_{\text{max}} \\
     \text{s.t.} \quad & x_{m-1, j} + p_{m, j} \leq x_{m, j}
         & \forall ~ j \in J; m \in (2, ..., |M|)\\
     & x_{m, j} + p_{m, j} \leq x_{m, k} \lor x_{m, k} + p_{m, k} \leq x_{m, j}
         & \forall ~ j \in J; k \in J, j \neq k\\
     & x_{|M|, j} + p_{|M|, j} \leq C_{\text{max}}
         & \forall ~ j \in J\\
     & x_{m, j} \geq 0 & \forall ~ j \in J; m \in M\\
     & z_{j, k} \in \{0, 1\} & \forall ~ j \in J; k \in J, j \neq k\\
 \end{align}
 $$

 You can compare this implementation to MILP solvers at the [end of the notebook](#bonus---milp-model).

In [1]:
import json

from bnbprob.pfssp import CallbackBnB, ExpBnB, LazyBnB, PermFlowShopLazy, plot_gantt
from bnbprob.pfssp.cython.heuristics import (
    local_search,
    neh_constructive,
    quick_constructive,
)
from bnbprob.pfssp.cython.permutation import Permutation

# from bnbprob.pfssp.cython.problem import PermFlowShopLazy as CyPermFlowShopLazy
from bnbprob.pfssp.cython.problem import PermFlowShopLazy as CyPermFlowShopLazy, FlowSolution
from bnbpy import configure_logfile

In [2]:
configure_logfile("pfssp-bench.log", mode="w")

In [23]:
with open("./../data/flow-shop/ta016.json", mode="r", encoding="utf8") as f:
    p = json.load(f)

In [35]:
import random

from bnbpy.node import Node

random.seed(42)


class BnB(ExpBnB):
    # def post_eval_callback(self, node: Node):
    #     if node.level >= node.problem.solution.n_jobs / 2:
    #         return super().post_eval_callback(node)

    def solution_callback(self, node: Node):
        pass
        new_sol = node.problem.local_search()
        if new_sol is not None:
            # General procedure in case is valid
            if new_sol.is_feasible() and new_sol.lb < node.solution.lb:
                node.set_solution(new_sol)
                node.check_feasible()
                self.set_solution(node)

    def fathom(self, node: Node):
        super().fathom(node)
        if node.parent is not None:
            node.parent.children.remove(node)
        del node.problem.solution
        node.problem.solution = None
        del node.problem
        node.problem = None
        del node

    def branch(self, node: Node):
        super().branch(node)
        if node.children:
            for child in node.children:
                child.parent = None
        del node.problem.solution
        node.problem.solution = None
        del node.problem
        node.problem = None
        del node

    def dequeue(self) -> Node:
        if self.explored % (len(p) * len(p[0]) / 2) == 0:
            pri, node = min(self.queue, key=lambda x: x[-1].lb)
            self.queue.remove((pri, node))
            return node
        # if self.explored % len(p) * 6 == 0:
        #     pri, node = random.choice(self.queue)
        #     # pri, node = min(self.queue, key=lambda x: x[-1].lb)
        #     self.queue.remove((pri, node))
        #     return node
        return super().dequeue()

In [36]:
problem_sm = PermFlowShopLazy.from_p(p, constructive='neh')
bnb_sm = BnB(eval_node="in", rtol=0.0001)

In [26]:
perm = Permutation.from_p(p)

In [27]:
FlowSolution(quick_constructive(problem_sm.solution.perm)).perm.calc_bound()

1505

In [28]:
problem_sm.quick_constructive().perm.calc_bound()

1505

In [29]:
print(problem_sm.solution.perm.calc_lb_2m())
print(perm.calc_lb_2m())
print(problem_sm.neh_constructive().perm.calc_bound())
print(neh_constructive(perm).calc_bound())
print(problem_sm.quick_constructive().perm.calc_bound())
print(quick_constructive(perm).calc_bound())
print(local_search(local_search(quick_constructive(perm))).calc_bound())

1282
1282
1471
1471
1511
1511
1488


In [30]:
import time

s = time.time()

for _ in range(100):
    # problem_sm.neh_constructive()
    # neh_constructive(perm)
    problem_sm.solution.perm.calc_lb_2m()
    # perm.calc_lb_2m()
    # perm.copy().push_job(0)
    # problem_sm.solution.copy().push_job(0)
    # problem_sm.solution.perm.sigma1.copy()

print(time.time() - s)

0.048972129821777344


In [31]:
0.18 / 0.06

3.0

In [37]:
sol_sm = bnb_sm.solve(
    problem_sm, maxiter=500000, timelimit=300
)
print(sol_sm)

Status: OPTIMAL | Cost: 1397 | LB: 1397


In [43]:
bnb_sm.queue[0][-1].problem.__dict__

{'solution': Status: INFEASIBLE | Cost: None | LB: 1553, 'constructive': 'neh'}

In [38]:
len(bnb_sm.queue)

164

In [ ]:
bnb_sm.root.children[2].children[3].children[4].children

[]

In [32]:
bnb_sm.queue[0][-1].solution.perm.lower_bound_2m()

2153

In [ ]:
# Save for unit testing

# p = [
#     [5, 9, 8, 10, 1],
#     [9, 3, 10, 1, 8],
#     [9, 4, 5, 8, 6],
#     [4, 8, 8, 7, 2]
# ]

In [ ]:
plot_gantt(sol_sm.sequence, dpi=120, seed=42, figsize=[8, 3])

In this implmentation lower bounds are computed by
the max of a single machine and
a two machine relaxations.

The bounds for single and two-machine problems are described
by Potts (1980), also implemented by Ladhari & Haouari (2005),
therein described as 'LB1' and 'LB5'.

The warmstart strategy is proposed by Palmer (1965).

## References

Ladhari, T., & Haouari, M. (2005). A computational study of
the permutation flow shop problem based on a tight lower bound.
Computers & Operations Research, 32(7), 1831-1847.

Potts, C. N. (1980). An adaptive branching rule for the permutation
flow-shop problem. European Journal of Operational Research, 5(1), 19-25.

Palmer, D. S. (1965). Sequencing jobs through a multi-stage process
in the minimum total time—a quick method of obtaining a near optimum.
Journal of the Operational Research Society, 16(1), 101-107

In [ ]:
import pyomo.environ as pyo

model = pyo.ConcreteModel()

# Sets for machines, jobs, horizon, and job sequences
model.M = pyo.Set(initialize=range(len(p[0])))
model.J = pyo.Set(initialize=range(len(p)))
model.K = pyo.Set(initialize=range(len(p)))

# Parameters
model.p = pyo.Param(model.J, model.M, initialize=lambda _, m, j: p[m][j])
model.V = pyo.Param(initialize=sum(pim for pi in p for pim in pi))

# Variables
model.x = pyo.Var(model.J, model.K, within=pyo.Binary)
model.h = pyo.Var(model.M, model.K, within=pyo.NonNegativeReals)
model.C = pyo.Var(within=pyo.NonNegativeReals)


# Constraints
def cstr_position(model, k):
    return sum(model.x[j, k] for j in model.J) == 1


def cstr_job(model, j):
    return sum(model.x[j, k] for k in model.K) == 1


def cstr_seq(model, m, k):
    if k == model.K.last():
        return pyo.Constraint.Skip
    return (
        model.h[m, k] + sum(model.p[j, m] * model.x[j, k] for j in model.J)
        <= model.h[m, k + 1]
    )


def cstr_precede(model, m, k):
    if m == model.M.last():
        return pyo.Constraint.Skip
    return (
        model.h[m, k] + sum(model.p[j, m] * model.x[j, k] for j in model.J)
        <= model.h[m + 1, k]
    )


def cstr_comp_precede(model, j, k):
    if j == k:
        return model.z[j, k] + model.z[k, j] == 0.0
    return model.z[j, k] + model.z[k, j] == 1.0


def cstr_total_time(model, m):
    k = model.K.last()
    return (
        model.h[m, k] + sum(model.p[j, m] * model.x[j, k] for j in model.J)
        <= model.C
    )


model.cstr_position = pyo.Constraint(model.K, rule=cstr_position)
model.cstr_job = pyo.Constraint(model.K, rule=cstr_job)
model.cstr_seq = pyo.Constraint(model.M, model.K, rule=cstr_seq)
model.cstr_precede = pyo.Constraint(model.M, model.K, rule=cstr_precede)
model.cstr_total_time = pyo.Constraint(model.M, rule=cstr_total_time)


# Objective
model.obj = pyo.Objective(expr=model.C, sense=pyo.minimize)

# HiGHS
# solver = pyo.SolverFactory('appsi_highs')
# solver.options['mip_heuristic_effort'] = 0.1
# solver.options['time_limit'] = 120
# solver.options['log_file'] = 'Highs.log'
# solver.solve(model, tee=True)

# Gurobi
solver = pyo.SolverFactory("gurobi", solver_io="python")
solver.options["Heuristics"] = 0.2
solver.options["Cuts"] = 2
solver.options["TimeLimit"] = 300
solver.solve(model, tee=True)

## Bonus - MILP Model

This is the usual Disjunctive MILP model as an alternative to compare performance.

### Position-based model

```python
import pyomo.environ as pyo

model = pyo.ConcreteModel()

# Sets for machines, jobs, horizon, and job sequences
model.M = pyo.Set(initialize=range(len(p[0])))
model.J = pyo.Set(initialize=range(len(p)))
model.K = pyo.Set(initialize=range(len(p)))

# Parameters
model.p = pyo.Param(model.J, model.M, initialize=lambda _, m, j: p[m][j])
model.V = pyo.Param(initialize=sum(pim for pi in p for pim in pi))

# Variables
model.x = pyo.Var(model.J, model.K, within=pyo.Binary)
model.h = pyo.Var(model.M, model.K, within=pyo.NonNegativeReals)
model.C = pyo.Var(within=pyo.NonNegativeReals)


# Constraints
def cstr_position(model, k):
    return sum(model.x[j, k] for j in model.J) == 1


def cstr_job(model, j):
    return sum(model.x[j, k] for k in model.K) == 1


def cstr_seq(model, m, k):
    if k == model.K.last():
        return pyo.Constraint.Skip
    return (
        model.h[m, k] + sum(model.p[j, m] * model.x[j, k] for j in model.J)
        <= model.h[m, k + 1]
    )


def cstr_precede(model, m, k):
    if m == model.M.last():
        return pyo.Constraint.Skip
    return (
        model.h[m, k] + sum(model.p[j, m] * model.x[j, k] for j in model.J)
        <= model.h[m + 1, k]
    )


def cstr_comp_precede(model, j, k):
    if j == k:
        return model.z[j, k] + model.z[k, j] == 0.0
    return model.z[j, k] + model.z[k, j] == 1.0


def cstr_total_time(model, m):
    k = model.K.last()
    return (
        model.h[m, k] + sum(model.p[j, m] * model.x[j, k] for j in model.J)
        <= model.C
    )


model.cstr_position = pyo.Constraint(model.K, rule=cstr_position)
model.cstr_job = pyo.Constraint(model.K, rule=cstr_job)
model.cstr_seq = pyo.Constraint(model.M, model.K, rule=cstr_seq)
model.cstr_precede = pyo.Constraint(model.M, model.K, rule=cstr_precede)
model.cstr_total_time = pyo.Constraint(model.M, rule=cstr_total_time)


# Objective
model.obj = pyo.Objective(expr=model.C, sense=pyo.minimize)

# HiGHS
solver = pyo.SolverFactory('appsi_highs')
solver.options['mip_heuristic_effort'] = 0.1
solver.options['time_limit'] = 120
solver.options['log_file'] = 'Highs.log'
solver.solve(model, tee=True)

# Gurobi
solver = pyo.SolverFactory("gurobi", solver_io="python")
solver.options["Heuristics"] = 0.2
solver.options["Cuts"] = 2
solver.options["TimeLimit"] = 120
solver.solve(model, tee=True)
```


### Disjunctive model

In experiments with 15 jobs and 8 machines, the Lazy implementation solved in 24s whereas with a time limit of 120s Gurobi could not reach the optimal solution (with a MIP gap of nearly 25%).

```python
import pyomo.environ as pyo

model = pyo.ConcreteModel()

# Sets for machines, jobs, horizon, and job sequences
model.M = pyo.Set(initialize=range(len(p[0])))
model.J = pyo.Set(initialize=range(len(p)))

# Parameters
model.p = pyo.Param(model.J, model.M, initialize=lambda _, m, j: p[m][j])
model.V = pyo.Param(initialize=sum(pim for pi in p for pim in pi))

# Variables
model.x = pyo.Var(model.J, model.M, within=pyo.NonNegativeReals)
model.z = pyo.Var(model.J, model.J, within=pyo.Binary)
model.C = pyo.Var(within=pyo.NonNegativeReals)


# Constraints
def cstr_seq(model, j, m):
    if m == model.M.last():
        return pyo.Constraint.Skip
    return model.x[j, m] + model.p[j, m] <= model.x[j, m + 1]


def cstr_precede(model, j, k, m):
    return model.x[j, m] + model.p[j, m] <= model.x[k, m] + model.V * (
        1 - model.z[j, k]
    )


def cstr_comp_precede(model, j, k):
    if j == k:
        return model.z[j, k] + model.z[k, j] == 0.0
    return model.z[j, k] + model.z[k, j] == 1.0


def cstr_total_time(model, j, m):
    return model.x[j, m] + model.p[j, m] <= model.C


model.cstr_seq = pyo.Constraint(model.J, model.M, rule=cstr_seq)
model.cstr_precede = pyo.Constraint(
    model.J, model.J, model.M, rule=cstr_precede
)
model.cstr_comp_precede = pyo.Constraint(
    model.J, model.J, rule=cstr_comp_precede
)
model.cstr_total_time = pyo.Constraint(model.J, model.M, rule=cstr_total_time)


# Objective
model.obj = pyo.Objective(expr=model.C, sense=pyo.minimize)

# HiGHS
solver = pyo.SolverFactory("appsi_highs")
solver.options["mip_heuristic_effort"] = 0.1
solver.options["time_limit"] = 120
solver.options["log_file"] = "Highs.log"
solver.solve(model, tee=True)

# Gurobi
solver = pyo.SolverFactory("gurobi", solver_io="python")
solver.options["Heuristics"] = 0.2
solver.options["Cuts"] = 2
solver.options["TimeLimit"] = 120
solver.solve(model, tee=True)
```

In [ ]:
import pyomo.environ as pyo

model = pyo.ConcreteModel()

# Sets for machines, jobs, horizon, and job sequences
model.M = pyo.Set(initialize=range(len(p[0])))
model.J = pyo.Set(initialize=range(len(p)))

# Parameters
model.p = pyo.Param(model.J, model.M, initialize=lambda _, m, j: p[m][j])
model.V = pyo.Param(initialize=sum(pim for pi in p for pim in pi))

# Variables
model.x = pyo.Var(model.J, model.M, within=pyo.NonNegativeReals)
model.z = pyo.Var(model.J, model.J, within=pyo.Binary)
model.C = pyo.Var(within=pyo.NonNegativeReals)


# Constraints
def cstr_seq(model, j, m):
    if m == model.M.last():
        return pyo.Constraint.Skip
    return model.x[j, m] + model.p[j, m] <= model.x[j, m + 1]


def cstr_precede(model, j, k, m):
    return model.x[j, m] + model.p[j, m] <= model.x[k, m] + model.V * (
        1 - model.z[j, k]
    )


def cstr_comp_precede(model, j, k):
    if j == k:
        return model.z[j, k] + model.z[k, j] == 0.0
    return model.z[j, k] + model.z[k, j] == 1.0


def cstr_total_time(model, j, m):
    return model.x[j, m] + model.p[j, m] <= model.C


model.cstr_seq = pyo.Constraint(model.J, model.M, rule=cstr_seq)
model.cstr_precede = pyo.Constraint(
    model.J, model.J, model.M, rule=cstr_precede
)
model.cstr_comp_precede = pyo.Constraint(
    model.J, model.J, rule=cstr_comp_precede
)
model.cstr_total_time = pyo.Constraint(model.J, model.M, rule=cstr_total_time)


# Objective
model.obj = pyo.Objective(expr=model.C, sense=pyo.minimize)

# HiGHS
# solver = pyo.SolverFactory("appsi_highs")
# solver.options["mip_heuristic_effort"] = 0.1
# solver.options["time_limit"] = 120
# solver.options["log_file"] = "Highs.log"
# solver.solve(model, tee=True)

# Gurobi
solver = pyo.SolverFactory("gurobi", solver_io="python")
solver.options["Heuristics"] = 0.2
solver.options["Cuts"] = 2
solver.options["TimeLimit"] = 120
solver.solve(model, tee=True)